In [1]:
# -------------------------------------------------------------
# 1. Imports and Setup
# -------------------------------------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# -------------------------------------------------------------
# 2. CNN Model Definition
# -------------------------------------------------------------
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 128 → 64
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)    # 64 → 32
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, 2)  # Binary: Authentic vs Tampered
        )

    def forward(self, x):
        return self.fc(self.conv(x))

# -------------------------------------------------------------
# 3. Load Dataset
# -------------------------------------------------------------
DATA_ROOT = r"C:\Users\Admin\OneDrive\Desktop\ResearchTrack\CASIA_FL_Project\data"

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

full_dataset = datasets.ImageFolder(DATA_ROOT, transform=transform)

# Split dataset: Train (75%), Test (20%), Eval (5%)
train_size = int(0.75 * len(full_dataset))
test_size = int(0.20 * len(full_dataset))
eval_size = len(full_dataset) - train_size - test_size

train_set, test_set, eval_set = random_split(full_dataset, [train_size, test_size, eval_size])

trainloader = DataLoader(train_set, batch_size=64, shuffle=True)
testloader = DataLoader(test_set, batch_size=64, shuffle=False)

print(f"✅ Dataset loaded successfully!")
print(f"Train: {len(train_set)} | Test: {len(test_set)} | Eval: {len(eval_set)}")

# -------------------------------------------------------------
# 4. Evaluation Function
# -------------------------------------------------------------
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"🔍 Accuracy  : {acc:.4f}")
    print(f"Precision : {precision:.4f}")
    print(f"Recall    : {recall:.4f}")
    print(f"F1 Score  : {f1:.4f}")

# -------------------------------------------------------------
# 5. Centralized Training
# -------------------------------------------------------------
model = SimpleCNN().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

EPOCHS = 15  # total epochs for centralized training
print("\n🚀 Starting Centralized Training...\n")

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in trainloader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(trainloader)
    print(f"🧠 Epoch [{epoch+1}/{EPOCHS}] - Loss: {avg_loss:.4f}")

    # Evaluate after each epoch
    print(f"\n📊 Evaluation after Epoch {epoch+1}:")
    evaluate_model(model, testloader)
    print("-" * 60)

print("\n✅ Centralized training completed successfully!")


Using device: cpu
✅ Dataset loaded successfully!
Train: 1290 | Test: 344 | Eval: 87

🚀 Starting Centralized Training...

🧠 Epoch [1/15] - Loss: 0.6919

📊 Evaluation after Epoch 1:
🔍 Accuracy  : 0.5727
Precision : 0.2863
Recall    : 0.5000
F1 Score  : 0.3641
------------------------------------------------------------
🧠 Epoch [2/15] - Loss: 0.6927

📊 Evaluation after Epoch 2:
🔍 Accuracy  : 0.5727
Precision : 0.2863
Recall    : 0.5000
F1 Score  : 0.3641
------------------------------------------------------------
🧠 Epoch [3/15] - Loss: 0.6903

📊 Evaluation after Epoch 3:
🔍 Accuracy  : 0.5727
Precision : 0.2863
Recall    : 0.5000
F1 Score  : 0.3641
------------------------------------------------------------
🧠 Epoch [4/15] - Loss: 0.6894

📊 Evaluation after Epoch 4:
🔍 Accuracy  : 0.5407
Precision : 0.5346
Recall    : 0.5351
F1 Score  : 0.5344
------------------------------------------------------------
🧠 Epoch [5/15] - Loss: 0.6891

📊 Evaluation after Epoch 5:
🔍 Accuracy  : 0.5698
Precisi